In [1]:
import numpy as np
from keras.utils import to_categorical

np.set_printoptions(precision=2)

Using TensorFlow backend.


In [2]:
X = np.array([
    [1, 2, 3],
    [20, 20, 10],
    [10, 20, 30],
    [25, 25, 26],
    
    [5, 10, 5],
    [5, 20, 15],
    [15, 30, 10],
    [33, 33, 34],
    
    [8, 9, 5],
    [10, 10, 15],
    [25, 25, 25],
    [30, 29, 28],
    
    [10, 10, 5],
    [8, 10, 9],
    [45, 5, 20],
    [85, 1, 1],
    
    [6, 10, 8],
    [10, 10, 30],
    [25, 25, 20],
    [82, 3, 4],
    
    [5, 1, 9],
    [10, 8, 20],
    [1, 2, 50],
    [35, 25, 31],
    
    [8, 6, 2],
    [30, 2, 4],
    [8, 35, 10],
    [29, 30, 31],
    
    [3, 5, 7],
    [33, 1, 9],
    [12, 13, 29],
    [3, 87, 1],
])

y = np.array([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]) - 1
y = to_categorical(y)

In [11]:
class OneVsAllClassificationNeuralNetwork:
    def __init__(self):
        self.layers = []
        
    def add_layer(self, units):
        self.layers.append({
            'units': units,
        })
    
    def initialize_weights(self):
        parameters = {}
        
        for i in range(len(self.layers)):            
            if i == 0:
                n_prev = self.X.shape[0]
            else:
                n_prev = self.layers[i - 1].get('units')
                
            i_str = str(i + 1)
            layer = self.layers[i]
            units = layer.get('units')
            
            parameters['W' + i_str] = np.random.rand(n_prev, units) * 0.01
            parameters['b' + i_str] = np.zeros((units, 1))
            
        return parameters
    
    def fit(self, X, y, epochs=1, learning_rate=0.01):
        self.X = X.T
        self.y = y.T
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.layers_len = len(self.layers)
        self.examples_len = self.X.shape[1]

        self.parameters = self.initialize_weights()
        
        for i in range(epochs):
            A = self.forward_propagate(self.X)
            
            if (i + 1) % 100 == 0 or i == 0:
                cost = self.compute_cost(A[self.layers_len])
                print('Epoch ' + str(i + 1) + ' cost: ' + str(cost))
            
            changes = self.backward_propagate(A)
            
            self.parameters = self.optimize(self.parameters, changes)
            
    def forward_propagate(self, X, debug=False):
        A = [X]
        
        for i in range(1, self.layers_len + 1):
            i_str = str(i)
            
            Z = np.dot(self.parameters['W' + i_str].T, A[i - 1]) + self.parameters['b' + i_str]
            A.append(self.sigmoid(Z))
        
        return A
    
    def backward_propagate(self, A):
        changes = {}
        dZs = {}
        
        for i in reversed(range(1, self.layers_len + 1)):
            i_str = str(i)
            current_A = A[i]
            
            if i == self.layers_len:
                dZ = A[self.layers_len] - self.y
            else:
                dZ = np.dot(self.parameters['W' + str(i + 1)], next_dZ) * (current_A * (1 - current_A))
            
            next_dZ = dZ
            changes['dW' + i_str] = (1 / self.examples_len) * np.dot(A[i - 1], dZ.T)
            changes['db' + i_str] = (1 / self.examples_len) * np.sum(dZ)
            
        return changes
    
    def optimize(self, parameters, changes):
        for i in range(1, self.layers_len + 1):
            i_str = str(i)
                        
            parameters['W' + i_str] -= self.learning_rate * changes['dW' + i_str]
            parameters['b' + i_str] -= self.learning_rate * changes['db' + i_str]
            
        return parameters
    
    def compute_cost(self, A):
        return (-1 / self.examples_len) * np.sum(self.y * np.log(A) + (1 - self.y) * np.log(1 - A))
    
    def predict(self, X, debug=False):
        return self.forward_propagate(X.T, debug)[-1]
    
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

In [20]:
model = OneVsAllClassificationNeuralNetwork()
model.add_layer(10)
model.add_layer(10)
model.add_layer(4)
model.fit(X, y, 3000, 0.1)

Epoch 1 cost: 2.8004635792963404
Epoch 100 cost: 2.2493364460599383
Epoch 200 cost: 2.2493350139954895
Epoch 300 cost: 2.2493328900527203
Epoch 400 cost: 2.249329559326416
Epoch 500 cost: 2.2493240671845403
Epoch 600 cost: 2.249314523357718
Epoch 700 cost: 2.2492969156110227
Epoch 800 cost: 2.249262083151928
Epoch 900 cost: 2.2491876489232743
Epoch 1000 cost: 2.2490166118812027
Epoch 1100 cost: 2.248606074717302
Epoch 1200 cost: 2.247630698622411
Epoch 1300 cost: 2.2454344331637826
Epoch 1400 cost: 2.2407537247493168
Epoch 1500 cost: 2.23131145818029
Epoch 1600 cost: 2.213153180836284
Epoch 1700 cost: 2.1767642187235317
Epoch 1800 cost: 2.0928603172887184
Epoch 1900 cost: 1.9057495121090438
Epoch 2000 cost: 1.679349901053416
Epoch 2100 cost: 1.5467200490415354
Epoch 2200 cost: 1.4806988323917496
Epoch 2300 cost: 1.490419234354262
Epoch 2400 cost: 1.4615471461432252
Epoch 2500 cost: 1.4420473626278238
Epoch 2600 cost: 1.4272517760184713
Epoch 2700 cost: 1.415076527379338
Epoch 2800 cost

In [26]:
X_test = np.array([[3, 4, 15], [20, 1, 25]])
model.predict(X_test, debug=True)

array([[0.61, 0.06],
       [0.34, 0.34],
       [0.05, 0.19],
       [0.  , 0.02]])